In [671]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score

In [672]:
with open('dataframes/df2015', 'rb') as f_2015:
    df2015 = pickle.load(f_2015)
f_2015.close()

with open('dataframes/df2016', 'rb') as f_2016:
    df2016 = pickle.load(f_2016)
f_2016.close()

with open('dataframes/df2017', 'rb') as f_2017:
    df2017 = pickle.load(f_2017)
f_2017.close()

with open('dataframes/df2018', 'rb') as f_2018:
    df2018 = pickle.load(f_2018)
f_2018.close()

with open('dataframes/df2019', 'rb') as f_2019:
    df2019 = pickle.load(f_2019)
f_2019.close()

with open('dataframes/df2020', 'rb') as f_2020:
    df2020 = pickle.load(f_2020)
f_2020.close()

with open('dataframes/df_all', 'rb') as f_all:
    df_all = pickle.load(f_all)
f_all.close()

with open('dataframes/df_feats', 'rb') as f_feats:
    df_feats = pickle.load(f_feats)
f_feats.close()

# Linear Regression
- The data in this set lends itself to linear regression since the happiness score is positively correlated with all its features as seen in the EDA steps. 
- GDP is the highest correlated features. 
- Since there are only 7 features, it's likely that we won't need to perform PCA on the dataset since there is no worthy feature to remove. These have all been engineered to be linearly correlated with our target (happiness). 
- Here, we will explore different ways regression can be applied to this dataset to make predictions.

### Question 1: What is the happines score of each country for 2020 given the 2020 features?
- we predicted for MOST countries based on SOME countries for this one

In [673]:
Y = df2020['Score'] # target 
X = df2020.iloc[:, 1:].drop(['Score'],axis='columns') # features 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=.2)
# print(train_x.shape)
# print(test_y.shape) # ok! 

In [674]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x.iloc[:, 1:])
test_x = scaler.transform(test_x.iloc[:, 1:])

In [675]:
lr_reg = LinearRegression()
lr_reg.fit(train_x, train_y) # keep the countries # index without it 
hyp = lr_reg.predict(test_x)

In [676]:
# metrics
res = test_y - hyp
r2_score = r2_score(test_y, hyp)
var = explained_variance_score(test_y, hyp)
mse = mean_squared_error(test_y, hyp)
print('R2 score...          \t', r2_score)
print('Variance explained...\t', var)
print('Mean squared error...\t', mse)
# DON'T TOUCH THIS # IT'S VERY GOOD

R2 score...          	 0.7674583920737953
Variance explained...	 0.7837391581941752
Mean squared error...	 0.2643099252837662


In [617]:
# tell me which countries were predicted and what they are

### Question 2: What is the happiness score of each country for 2020 given the the data from past years (2015-2019)?
- This is kind of a hot mess
- See how Ariana does this?

In [627]:
Y = df_feats['Score'] # target 
X = df_feats.iloc[:, 1:].drop(['Score'],axis='columns') # features 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=.2)
# print(train_x.shape)
# print(test_y.shape) # ok! 

In [628]:
# scaler = StandardScaler()
# train_x = scaler.fit_transform(train_x.iloc[:, 1:])
# test_x = scaler.fit(test_x.iloc[:, 1:])

In [629]:
# lr_reg = LinearRegression()
# lr_reg = GridSearchCV(lr, param_grid={}, n_jobs = 5)
# lr_reg.fit(train_x.iloc[:, 2:], train_y) # keep the countries # index without it 
# hyp = lr_reg.predict(test_x.iloc[:, 2:])

In [ ]:
# r2_score = r2_score(test_y, hyp)
# var = explained_variance_score(test_y, hyp)
# mse = mean_squared_error(test_y, hyp)
# print('R2 score...\t', r2_score)
# print('Variance explained...\t', var)
# print('Mean squared error...\t', mse)

### Question 3: Given that we predict the each feature for 2020, can we obtain a valid happiness score?
- We will predict the GDP for 2020 given 2015-2019, Health Life Expectancy for 2020 given 2015-2019, etc. 
- Once those features are predicted, we will predict the happiness score! 

In [630]:
print(df2015.shape)
print(df2020.shape)

(158, 10)
(153, 9)


In [ ]:
# GDP per capita

In [ ]:
# Healthy life expectancy

In [ ]:
# Social support

In [ ]:
# Freedom to make life choices

In [ ]:
# Perceptions of corruption

In [ ]:
# Generosity

# References
- [Happiness Data Analysis 2015-2019](https://www.kaggle.com/kojisera/happiness-data-analysis-2015-2019) - See top happy 30 countries scatter plot for 2015-2019